In [244]:
# Include necessary imports
import os
import torch 
import torch.nn as nn
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from music21 import *
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from transformers import T5ForConditionalGeneration, T5Tokenizer
import random

In [245]:
# Pre Process Data
def get_pitch_class(note):
    return note % 12

def find_matching_octave_note(df):
    bass_line = df.iloc[3].values
    last_bass_note = bass_line[-1]
    return last_bass_note

def explore_for_lowest_tonic(df, pitch_class):
    bass_notes = df.iloc[3].values  
    matching_notes = [note for note in bass_notes if get_pitch_class(note) == pitch_class]
    if matching_notes:
        return min(matching_notes) 
    else:
        return bass_notes[0]

def detect_tonic(df):
    candidate_note = find_matching_octave_note(df)
    pitch_class = get_pitch_class(candidate_note)
    true_tonic_note = explore_for_lowest_tonic(df, pitch_class)
    return true_tonic_note

def key_transposition(df):
    tonic_note = detect_tonic(df)
    transpose_val = 48 - tonic_note
    df = (df + transpose_val).clip(lower=0, upper=127)
    return df


def normalize_df(df):
    X_std = df / 127
    return X_std

folder_path = 'Data/'
test = []
train = []
validation = []
for dirname in os.listdir(folder_path):
    if dirname != '.DS_Store':
        for filename in os.listdir(folder_path + dirname):
            if filename != '.ipynb_checkpoints':
                df = pd.read_csv(folder_path + dirname + '/' + filename)
                transposed_df = key_transposition(df.transpose())
                normalized_df = normalize_df(transposed_df)
                if dirname == 'test':
                    test.append(normalized_df)
                if dirname == 'train':
                    train.append(normalized_df)
                if dirname == 'valid':
                    validation.append(normalized_df)

# Model

In [282]:
class Model(torch.nn.Module):
    def __init__(self, input_size, output_size, hidden_dim=100, n_layers=2, dropout_rate=.5):
        super(Model, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.lstm = torch.nn.LSTM(input_size, hidden_dim, n_layers, batch_first=True, dropout=dropout_rate)
        self.fc = torch.nn.Linear(hidden_dim, output_size * 128)
        self.dropout = torch.nn.Dropout(dropout_rate)

    def init_hidden(self, batch_size):
        return (torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(next(self.parameters()).device),
                torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(next(self.parameters()).device))

    def forward(self, x, target=None, hidden=None, teacher_forcing=.5):
        batch_size, seq_len, _ = x.size()
        if hidden is None:
            hidden = self.init_hidden(batch_size)
        lstm_input = x[:, 0:1, :]

        outputs = []

        for t in range(seq_len):
            lstm_output, hidden = self.lstm(lstm_input, hidden)
            lstm_output_step = lstm_output[:, -1, :] 
            model_output_step = self.fc(lstm_output_step)
            model_output_step = model_output_step.view(batch_size, 3, 128)

            outputs.append(model_output_step.unsqueeze(1))
            
            if target is not None and random.random() < teacher_forcing:
                next_harmony = target[:, t:t+1, :]
            else:
                next_harmony = model_output_step.argmax(dim=-1).unsqueeze(1)

            next_harmony = next_harmony.view(batch_size, 1, 3)

            if t + 1 < seq_len:
                melody_at_next_step = x[:, t + 1:t + 2, :1]  
                lstm_input = torch.cat((melody_at_next_step, next_harmony), dim=2) 

        output = torch.cat(outputs, dim=1)
        return output

# Train

In [293]:
def train_model(model, optimizer, criterion, num_epochs):
    model.train()
    for song_index, song in enumerate(train[:150]):
        print(f"Training on song {song_index + 1}")
        
        melody = torch.tensor(song.iloc[0].values.reshape(-1, 1), dtype=torch.float32).unsqueeze(0).reshape(1, song.shape[1], 1)
        harmonies = torch.tensor(song.iloc[1:].values.T, dtype=torch.float32).unsqueeze(0)
        harmonies_with_zero = torch.zeros(1, song.shape[1], 3)
        melody_with_empty_harmonies = torch.cat((melody, harmonies_with_zero), dim = -1)
        harmonies_for_loss = harmonies_to_class(harmonies)
       
        
        for epoch in range(num_epochs):
            optimizer.zero_grad()
            ratio = .9 - (.9 - .1) * (epoch / num_epochs)
            ratio = max(.1, ratio)
            output = model(x=melody_with_empty_harmonies, target=harmonies, teacher_forcing=ratio)
            output = output.reshape(-1, 128)
            harmonies_for_loss = harmonies_for_loss.reshape(-1)
#             with torch.no_grad():
#                 print(f"Targets    : {harmonies_for_loss[:50]}")
#                 print(f"Predictions: {output[:50].argmax(dim=1)}")
            loss = criterion(output, harmonies_for_loss)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            if (epoch + 1) % 10 == 0:
                print(f"Song {song_index + 1}, Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}")
                    

        model.eval()
        total_val_loss = 0
        

        
        # validation songs
        with torch.no_grad():
            for val_song in validation:
                val_melody = torch.tensor(val_song.iloc[0].values.reshape(-1, 1), dtype=torch.float32).unsqueeze(0).reshape(1, val_song.shape[1], 1)
                val_harmonies = torch.tensor(val_song.iloc[1:].values.T, dtype=torch.float32).unsqueeze(0)
                val_harmonies = harmonies_to_class(val_harmonies)
                val_harmonies_with_zero = torch.zeros(1, val_song.shape[1], 3)
                val_melody_with_empty_harmonies = torch.cat((val_melody, val_harmonies_with_zero), dim=-1)
                
                val_output = model(val_melody_with_empty_harmonies)
                val_output = val_output.reshape(-1, 128)
                val_harmonies = val_harmonies.reshape(-1)
                
                val_loss = criterion(val_output, val_harmonies)
                total_val_loss += val_loss.item()

        average_val_loss = total_val_loss / len(validation)
        print(f"Validation Loss after song {song_index + 1}: {average_val_loss}")

In [ ]:
def harmonies_to_class(harmonies):
    harmonies_classes = torch.round(harmonies * 127).long()  
    return harmonies_classes

criterion = torch.nn.CrossEntropyLoss()
num_epochs = 200

model = Model(4, 3)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-5)
train_model(model, optimizer, criterion, num_epochs)

Training on song 1
Song 1, Epoch 10/200, Loss: 4.826527118682861
Song 1, Epoch 20/200, Loss: 4.806944847106934
Song 1, Epoch 30/200, Loss: 4.763866901397705
Song 1, Epoch 40/200, Loss: 4.711582660675049
Song 1, Epoch 50/200, Loss: 4.601804733276367
Song 1, Epoch 60/200, Loss: 4.397435665130615
Song 1, Epoch 70/200, Loss: 4.112029552459717
Song 1, Epoch 80/200, Loss: 3.750270128250122
Song 1, Epoch 90/200, Loss: 3.4394004344940186
Song 1, Epoch 100/200, Loss: 3.171048879623413
Song 1, Epoch 110/200, Loss: 2.909698486328125
Song 1, Epoch 120/200, Loss: 2.715970993041992
Song 1, Epoch 130/200, Loss: 2.5552196502685547
Song 1, Epoch 140/200, Loss: 2.4439637660980225
Song 1, Epoch 150/200, Loss: 2.3771116733551025
Song 1, Epoch 160/200, Loss: 2.277440309524536
Song 1, Epoch 170/200, Loss: 2.2265398502349854
Song 1, Epoch 180/200, Loss: 2.1913485527038574
Song 1, Epoch 190/200, Loss: 2.177070140838623
Song 1, Epoch 200/200, Loss: 2.149001121520996
Validation Loss after song 1: 4.198215680244

Song 9, Epoch 40/200, Loss: 2.623589038848877
Song 9, Epoch 50/200, Loss: 2.5097668170928955
Song 9, Epoch 60/200, Loss: 2.381617784500122
Song 9, Epoch 70/200, Loss: 2.2789435386657715
Song 9, Epoch 80/200, Loss: 2.2323780059814453
Song 9, Epoch 90/200, Loss: 2.184739828109741
Song 9, Epoch 100/200, Loss: 2.153258800506592
Song 9, Epoch 110/200, Loss: 2.1351966857910156
Song 9, Epoch 120/200, Loss: 2.116194486618042
Song 9, Epoch 130/200, Loss: 2.0920557975769043
Song 9, Epoch 140/200, Loss: 2.0908734798431396
Song 9, Epoch 150/200, Loss: 2.086055278778076
Song 9, Epoch 160/200, Loss: 2.0667037963867188
Song 9, Epoch 170/200, Loss: 2.055654525756836
Song 9, Epoch 180/200, Loss: 2.0593760013580322
Song 9, Epoch 190/200, Loss: 2.046806573867798
Song 9, Epoch 200/200, Loss: 2.043516159057617
Validation Loss after song 9: 3.7065905729929605
Training on song 10
Song 10, Epoch 10/200, Loss: 2.2363474369049072
Song 10, Epoch 20/200, Loss: 2.154874324798584
Song 10, Epoch 30/200, Loss: 2.1041

Song 17, Epoch 40/200, Loss: 3.3692872524261475
Song 17, Epoch 50/200, Loss: 3.1634950637817383
Song 17, Epoch 60/200, Loss: 2.9883310794830322
Song 17, Epoch 70/200, Loss: 2.853451728820801
Song 17, Epoch 80/200, Loss: 2.72556734085083
Song 17, Epoch 90/200, Loss: 2.6358566284179688
Song 17, Epoch 100/200, Loss: 2.5418763160705566
Song 17, Epoch 110/200, Loss: 2.4636263847351074
Song 17, Epoch 120/200, Loss: 2.4103949069976807
Song 17, Epoch 130/200, Loss: 2.367213487625122
Song 17, Epoch 140/200, Loss: 2.3330421447753906
Song 17, Epoch 150/200, Loss: 2.308600902557373
Song 17, Epoch 160/200, Loss: 2.2889838218688965
Song 17, Epoch 170/200, Loss: 2.275946855545044
Song 17, Epoch 180/200, Loss: 2.2669782638549805
Song 17, Epoch 190/200, Loss: 2.2564806938171387
Song 17, Epoch 200/200, Loss: 2.244575023651123
Validation Loss after song 17: 3.9908623817639475
Training on song 18
Song 18, Epoch 10/200, Loss: 3.5836379528045654
Song 18, Epoch 20/200, Loss: 3.3697292804718018
Song 18, Epoch

Song 25, Epoch 40/200, Loss: 2.291566848754883
Song 25, Epoch 50/200, Loss: 2.226774215698242
Song 25, Epoch 60/200, Loss: 2.194234848022461
Song 25, Epoch 70/200, Loss: 2.168586015701294
Song 25, Epoch 80/200, Loss: 2.1530163288116455
Song 25, Epoch 90/200, Loss: 2.13523530960083
Song 25, Epoch 100/200, Loss: 2.121950149536133
Song 25, Epoch 110/200, Loss: 2.1156392097473145
Song 25, Epoch 120/200, Loss: 2.1084704399108887
Song 25, Epoch 130/200, Loss: 2.105189085006714
Song 25, Epoch 140/200, Loss: 2.1019062995910645
Song 25, Epoch 150/200, Loss: 2.0896739959716797
Song 25, Epoch 160/200, Loss: 2.086453914642334
Song 25, Epoch 170/200, Loss: 2.0949783325195312
Song 25, Epoch 180/200, Loss: 2.097369432449341
Song 25, Epoch 190/200, Loss: 2.0895848274230957
Song 25, Epoch 200/200, Loss: 2.0840823650360107
Validation Loss after song 25: 3.77374224173717
Training on song 26
Song 26, Epoch 10/200, Loss: 2.638803243637085
Song 26, Epoch 20/200, Loss: 2.4735803604125977
Song 26, Epoch 30/20

Song 33, Epoch 30/200, Loss: 2.5470468997955322
Song 33, Epoch 40/200, Loss: 2.3948047161102295
Song 33, Epoch 50/200, Loss: 2.243260383605957
Song 33, Epoch 60/200, Loss: 2.168640375137329
Song 33, Epoch 70/200, Loss: 2.125826835632324
Song 33, Epoch 80/200, Loss: 2.0666584968566895
Song 33, Epoch 90/200, Loss: 2.045847177505493
Song 33, Epoch 100/200, Loss: 2.0207386016845703
Song 33, Epoch 110/200, Loss: 2.0069022178649902
Song 33, Epoch 120/200, Loss: 1.9908298254013062
Song 33, Epoch 130/200, Loss: 1.9825571775436401
Song 33, Epoch 140/200, Loss: 1.9723107814788818
Song 33, Epoch 150/200, Loss: 1.9701025485992432
Song 33, Epoch 160/200, Loss: 1.9617831707000732
Song 33, Epoch 170/200, Loss: 1.9570281505584717
Song 33, Epoch 180/200, Loss: 1.949556827545166
Song 33, Epoch 190/200, Loss: 1.9446098804473877
Song 33, Epoch 200/200, Loss: 1.9563541412353516
Validation Loss after song 33: 3.935076872507731
Training on song 34
Song 34, Epoch 10/200, Loss: 4.0492963790893555
Song 34, Epoc

...

Song 72, Epoch 200/200, Loss: 2.2868330478668213
Validation Loss after song 72: 4.267060237053113
Training on song 73
Song 73, Epoch 10/200, Loss: 2.7836363315582275
Song 73, Epoch 20/200, Loss: 2.694728136062622
Song 73, Epoch 30/200, Loss: 2.692486047744751
Song 73, Epoch 40/200, Loss: 2.49710750579834
Song 73, Epoch 50/200, Loss: 2.4026029109954834
Song 73, Epoch 60/200, Loss: 2.3677501678466797
Song 73, Epoch 70/200, Loss: 2.3535423278808594
Song 73, Epoch 80/200, Loss: 2.171616315841675
Song 73, Epoch 90/200, Loss: 2.2736146450042725
Song 73, Epoch 100/200, Loss: 2.131622076034546
Song 73, Epoch 110/200, Loss: 2.0532519817352295
Song 73, Epoch 120/200, Loss: 2.0298566818237305
Song 73, Epoch 130/200, Loss: 2.057633399963379
Song 73, Epoch 140/200, Loss: 1.9777584075927734
Song 73, Epoch 150/200, Loss: 1.9810917377471924
Song 73, Epoch 160/200, Loss: 1.9366474151611328
Song 73, Epoch 170/200, Loss: 1.9486347436904907
Song 73, Epoch 180/200, Loss: 1.909778118133545
Song 73, Epoch 19

Song 80, Epoch 190/200, Loss: 2.091078281402588
Song 80, Epoch 200/200, Loss: 2.104550838470459
Validation Loss after song 80: 4.0289973601316795
Training on song 81
Song 81, Epoch 10/200, Loss: 2.4955663681030273
Song 81, Epoch 20/200, Loss: 2.3126351833343506
Song 81, Epoch 30/200, Loss: 2.1751694679260254
Song 81, Epoch 40/200, Loss: 2.0919086933135986
Song 81, Epoch 50/200, Loss: 2.045772075653076
Song 81, Epoch 60/200, Loss: 2.019686698913574
Song 81, Epoch 70/200, Loss: 2.001042127609253
Song 81, Epoch 80/200, Loss: 1.9776257276535034
Song 81, Epoch 90/200, Loss: 1.978487253189087
Song 81, Epoch 100/200, Loss: 1.9647663831710815
Song 81, Epoch 110/200, Loss: 1.9405276775360107
Song 81, Epoch 120/200, Loss: 1.9516757726669312
Song 81, Epoch 130/200, Loss: 1.947992205619812
Song 81, Epoch 140/200, Loss: 1.9400615692138672
Song 81, Epoch 150/200, Loss: 1.9335591793060303
Song 81, Epoch 160/200, Loss: 1.93316650390625
Song 81, Epoch 170/200, Loss: 1.9405183792114258
Song 81, Epoch 18

Song 88, Epoch 180/200, Loss: 1.5500725507736206
Song 88, Epoch 190/200, Loss: 1.5354492664337158
Song 88, Epoch 200/200, Loss: 1.5201953649520874
Validation Loss after song 88: 3.9015744160383177
Training on song 89
Song 89, Epoch 10/200, Loss: 2.370335578918457
Song 89, Epoch 20/200, Loss: 2.3252992630004883
Song 89, Epoch 30/200, Loss: 2.2610843181610107
Song 89, Epoch 40/200, Loss: 2.217198133468628
Song 89, Epoch 50/200, Loss: 2.169635534286499
Song 89, Epoch 60/200, Loss: 2.1327409744262695
Song 89, Epoch 70/200, Loss: 2.094726324081421
Song 89, Epoch 80/200, Loss: 2.0783705711364746
Song 89, Epoch 90/200, Loss: 2.029768705368042
Song 89, Epoch 100/200, Loss: 1.9850338697433472
Song 89, Epoch 110/200, Loss: 1.9208990335464478
Song 89, Epoch 120/200, Loss: 1.9091943502426147
Song 89, Epoch 130/200, Loss: 1.902743935585022
Song 89, Epoch 140/200, Loss: 1.8767553567886353
Song 89, Epoch 150/200, Loss: 1.9082046747207642
Song 89, Epoch 160/200, Loss: 1.8835680484771729
Song 89, Epoch

Song 96, Epoch 170/200, Loss: 2.03524112701416
Song 96, Epoch 180/200, Loss: 2.0244314670562744
Song 96, Epoch 190/200, Loss: 2.013171911239624
Song 96, Epoch 200/200, Loss: 2.003037214279175
Validation Loss after song 96: 4.180684117170481
Training on song 97
Song 97, Epoch 10/200, Loss: 2.9567224979400635
Song 97, Epoch 20/200, Loss: 2.878098726272583
Song 97, Epoch 30/200, Loss: 2.84533953666687
Song 97, Epoch 40/200, Loss: 2.7965328693389893
Song 97, Epoch 50/200, Loss: 2.7421576976776123
Song 97, Epoch 60/200, Loss: 2.706732749938965
Song 97, Epoch 70/200, Loss: 2.6619057655334473
Song 97, Epoch 80/200, Loss: 2.6532020568847656
Song 97, Epoch 90/200, Loss: 2.53393292427063
Song 97, Epoch 100/200, Loss: 2.48181414604187
Song 97, Epoch 110/200, Loss: 2.4497644901275635
Song 97, Epoch 120/200, Loss: 2.4182989597320557
Song 97, Epoch 130/200, Loss: 2.3845062255859375
Song 97, Epoch 140/200, Loss: 2.355186939239502
Song 97, Epoch 150/200, Loss: 2.339146375656128
Song 97, Epoch 160/200,

Song 104, Epoch 140/200, Loss: 2.0583291053771973
Song 104, Epoch 150/200, Loss: 2.0571961402893066
Song 104, Epoch 160/200, Loss: 2.023102045059204
Song 104, Epoch 170/200, Loss: 2.0185463428497314
Song 104, Epoch 180/200, Loss: 1.96644127368927
Song 104, Epoch 190/200, Loss: 1.936616063117981
Song 104, Epoch 200/200, Loss: 1.952702522277832
Validation Loss after song 104: 4.051210161967155
Training on song 105
Song 105, Epoch 10/200, Loss: 2.62518310546875
Song 105, Epoch 20/200, Loss: 2.5036051273345947
Song 105, Epoch 30/200, Loss: 2.4246857166290283
Song 105, Epoch 40/200, Loss: 2.359278678894043
Song 105, Epoch 50/200, Loss: 2.3085711002349854
Song 105, Epoch 60/200, Loss: 2.281064510345459
Song 105, Epoch 70/200, Loss: 2.1984739303588867
Song 105, Epoch 80/200, Loss: 2.1744446754455566


# Hyperparameter Tuning

In [ ]:
learning = [0.01, .001]
n_layers= [1,2,3]
hidden_dim = [20, 40, 50]
epochs= [5000, 10000]
best_loss = float('inf')
best_params = {}

for LR in learning:
    for n_layer in n_layers:
        for epoch in epochs:
            for dims in hidden_dim:
                print(f"Training with LR={LR} and n_layers={n_layer} and epochs={epoch} and hidden_dims={dims}")
                model = Model(input_size=1, output_size=harmonies.shape[2], n_layers=n_layer, hidden_dim=dims)
                optimizer = torch.optim.Adam(model.parameters(), lr=LR)
                train_model(model, melody, harmonies, optimizer, criterion, epoch)
                with torch.no_grad():
                    output = model(melody)
                    loss = criterion(output, harmonies)
                    print(f"Final Loss: {loss.item()}")        
                # Keep track of the best model (with lowest loss)
                if loss.item() < best_loss:
                    best_loss = loss.item()
                    best_params = {'learning': LR, 'n_layers': n_layer, 'epochs': epoch, 'hidden_dim': dims}
print("BEST: ", best_params)

In [229]:
def inverse_df(df):
    X_scaled = df * 127
    return X_scaled

def midi_to_note_melody(part):
    result = stream.Part()
    count = 1
    prev = round(part[0])
    for i in range(1, len(part)):
        pitch = part[i]
        curr = round(pitch)
        if curr == prev:
            count += 1
        else:
            this_note = note.Note(prev, quarterLength=count/4)
            result.append(this_note)
            count = 1
        prev = curr
    this_note = note.Note(prev, quarterLength=count/4)
    result.append(this_note)
    return result

def midi_to_note_harmonies(part):
    probabilities = torch.softmax(torch.tensor(part), dim=-1).numpy()
#     print(probabilities)
    result = stream.Part()
    count = 1
    predicted_notes = np.argmax(part, axis=1)
    print(predicted_notes)
    prev = predicted_notes[0]
    for i in range(1, len(part)):
        curr = predicted_notes[i]
        if curr == prev:
            count += 1
        else:
            this_note = note.Note(prev, quarterLength=count/4)
            result.append(this_note)
            count = 1
        prev = curr
    this_note = note.Note(prev, quarterLength=count/4)
    result.append(this_note)
    return result

def output_to_sheet_music(melody, result):
    result_numpy = result.squeeze(0).detach().numpy()
    melody = inverse_df(melody).squeeze()
    inversed = result_numpy.transpose(1, 0, 2) 
#     inversed = inverse_df(result_numpy.T) # for reg harmonies
    
    score = stream.Score()
    melody_part = midi_to_note_melody(melody.numpy())
    
    alto_notes = inversed[0]
    tenor_notes = inversed[1]
    bass_notes = inversed[2]  
    
    alto_part = midi_to_note_harmonies(alto_notes)
    tenor_part = midi_to_note_harmonies(tenor_notes)
    bass_part = midi_to_note_harmonies(bass_notes)

    score.append(melody_part)
    score.append(alto_part)
    score.append(tenor_part)
    score.append(bass_part)
    score.show('midi')
    score.write('musicxml', 'output.xml')

In [292]:
test_song = test[3]
melody = test_song.iloc[0]

melody = torch.tensor(test_song.iloc[0].values, dtype=torch.float32).unsqueeze(0).unsqueeze(-1)
harmonies = torch.tensor(test_song.iloc[1:].values.T, dtype=torch.float32).unsqueeze(0)
harmonies_with_zero = torch.zeros(1, test_song.shape[1], 3)

model_input = torch.cat((melody, harmonies_with_zero), dim = -1)
result = model(x=model_input)
output_to_sheet_music(melody, result)

[67 67 67 72 72 72 72 72 72 72 72 72 72 72 72 72 72 72 72 72 72 72 72 72
 72 72 72 72 72 72 72 72 72 72 72 72 72 72 72 72 72 72 72 72 72 72 72 72
 72 72 72 72 72 72 72 72 72 72 72 72 72 72 72 72 72 72 72 72 72 72 72 72
 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67
 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67
 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67
 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67
 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67
 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67
 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67
 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67
 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67
 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67
 67 67 67 67 67 67 67 67 67 67 67 67]
[67 67 67 67 

In [104]:
# Finetune (hyperparameters, move around test data (refer to notes), etc)

In [ ]:
# Test with new data + evaluate

In [ ]:
# Make any other changes

In [ ]:
# Sheet music + audio (musicAI)

In [ ]:
# Create new models if time permits (follow steps 3 - 7)

In [ ]:
# Compare models

In [ ]:
# Front end ** if time permits
# - Interactive sheet music
# - musescore front end??

In [ ]:
# real one and generated compare
# train on all songs + test on a different song
# measure the test loss not just the training loss

In [ ]:
[60, 0, 0, 0] -> [60, 70, 70, 70] -> [61, ]